In [6]:
from sklearn.datasets import make_circles
from sklearn.model_selection import train_test_split

X, y = make_circles(n_samples=1_000, factor=0.3, noise=0.05, random_state=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)




In [9]:
from sklearn.metrics.pairwise import rbf_kernel



(750, 750)

In [12]:
rbf_kernel(X_train)

array([[1.        , 0.73451141, 0.63319839, ..., 0.61076305, 0.14534005,
        0.3783166 ],
       [0.73451141, 1.        , 0.97782212, ..., 0.96092166, 0.48893632,
        0.55593275],
       [0.63319839, 0.97782212, 1.        , ..., 0.99685392, 0.54987154,
        0.47558138],
       ...,
       [0.61076305, 0.96092166, 0.99685392, ..., 1.        , 0.54187099,
        0.43166512],
       [0.14534005, 0.48893632, 0.54987154, ..., 0.54187099, 1.        ,
        0.43127951],
       [0.3783166 , 0.55593275, 0.47558138, ..., 0.43166512, 0.43127951,
        1.        ]])

In [16]:
from sklearn.decomposition import PCA, KernelPCA




In [17]:
X_train_kernel_pca.shape

(750, 67)

array([[0.71653131, 0.51341712, 1.        ],
       [0.51341712, 0.71653131, 0.36787944],
       [0.71653131, 0.51341712, 1.        ]])

# Isolation Kernal from R

In [117]:
import numpy as np
from sklearn.utils import check_random_state
from sklearn.metrics.pairwise import pairwise_distances

def IKFeature(data, Sdata=None, psi=64, t=200, Sp=True):
    if Sdata is None:
        Sdata = data.copy()

    sizeS = Sdata.shape[0]
    sizeN = data.shape[0]
    Feature = None

    for _ in range(t):
        subIndex = check_random_state(None).choice(sizeS, psi, replace=False)
        
        tdata = Sdata[subIndex, :]
        distances = pairwise_distances(tdata, data, metric='euclidean')
        #print(distances)
        nn_indices = np.argmin(distances, axis=0)
        OneFeature = np.zeros((sizeN, psi), dtype=int)
        OneFeature[np.arange(sizeN), nn_indices] = 1
        
        if Feature is None:
            Feature = OneFeature  # Initialize Feature with OneFeature in the first iteration
        else:
            Feature = np.hstack((Feature, OneFeature))  # Add OneFeature to Feature vertically


    return Feature  # sparse matrix
 


def IKSimilarity(data, Sdata=None, psi=64, t=200):
    Feature = IKFeature(data, Sdata, psi, t)
    SimMatrix = np.dot(Feature, Feature.T) / t
    return SimMatrix

In [114]:
# Compute SIKFeatures (sparse matrix)
SIKFeatures = IKFeature(data[:5,], psi=128, t=2)

[3 1 0 0 2]
[0 1 2 3 4] [3 1 0 0 2]
[1 2 0 0 3]
[0 1 2 3 4] [1 2 0 0 3]


In [115]:
SIKFeatures

array([[0, 0, 0, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 1]])

In [118]:
from sklearn.datasets import load_iris


# Load iris dataset
iris = load_iris()
data = iris.data[:, :4]  # Selecting first four columns

# Compute SIKFeatures (sparse matrix)
IKFeatures = IKFeature(data, psi=4, t=200)



In [123]:
IKSimilarity_my(data, psi=4, t=200)

(150, 150)

In [125]:
import numpy as np

# Assuming y_train and y_test are numpy arrays
# Here's a sample creation of y_train and y_test for demonstration purposes
y_train = np.array([1, 2, 3])
y_test = np.array([4, 5, 6])

# Stack y_train and y_test horizontally
y_combined = 

print("Combined Array:")
print(y_combined)


Combined Array:
[[1]
 [2]
 [3]
 [4]
 [5]
 [6]]
